# Handling Exceptions — Practice (Advanced, but not too much)

What we’ll practice:
- Catching **specific** exceptions (not overbroad `except Exception` unless you re-raise).
- Using `try/except/else/finally` appropriately.
- Re-raising and adding context.
- `contextlib.suppress` for ignore-and-continue cases.
- Simple retry patterns and resource cleanup.

Each task has a `# TODO` cell and an assert cell. Run the asserts to verify.

### Test Helpers (for asserts)
Small helpers to assert exceptions and messages.

In [1]:
def assert_raises(exc_type, func, *args, **kwargs):
    try:
        func(*args, **kwargs)
    except exc_type as e:
        return e
    except Exception as e:
        raise AssertionError(f"Expected {exc_type.__name__}, got {type(e).__name__}: {e}")
    else:
        raise AssertionError(f"Expected {exc_type.__name__} to be raised, but no exception occurred.")

def assert_message_contains(exc, text):
    assert text in str(exc), f"Message does not contain {text!r}: {exc}"

## 1) Safe Int Conversion with Fallback
Implement `safe_int(s, default=None)`:
- Return `int(s)` when possible.
- If `s` is a string/number but not convertible (e.g., `'x'`), return `default`.
- If `s` is the wrong type (e.g., `None`, list), raise `TypeError('s must be str or number')`.

In [2]:
# TODO: implement safe_int
def safe_int(s, default=None):
    try:
        return int(s)
    except ValueError:
        return default
    except TypeError:
        raise TypeError('s must be str or number')

safe_int('10')

10

In [3]:
assert safe_int('10') == 10
assert safe_int('x', -1) == -1
_e = assert_raises(TypeError, safe_int, None)
assert_message_contains(_e, 'str or number')
print('OK - 1')

OK - 1


## 2) Read Key with Type Check (using `else`)
Implement `read_key(mapping, key, required_type=None, default=None)`:
- Return `mapping[key]` if present.
- On missing key, return `default`.
- If `required_type` is given and value isn’t an instance, raise `TypeError('wrong type for key')`.
- **Fix included:** treat `bool` as **not** `int` when `required_type` is `int`.
- Use `try/except ... else:` so the type check runs only when the lookup succeeds.

In [4]:
# TODO: implement read_key (fixed for bool vs int)
def read_key(mapping, key, required_type=None, default=None):
    try:
        val = mapping[key]
    except KeyError:
        return default
    else:
        if required_type is not None:
            if required_type is int and isinstance(val, bool):
                raise TypeError('wrong type for key')
            if not isinstance(val, required_type):
                raise TypeError('wrong type for key')
        return val

read_key({'a': 1}, 'a')

1

In [5]:
d = {'port': 8080, 'debug': True}
assert read_key(d, 'port', int) == 8080
assert read_key(d, 'missing', default='N/A') == 'N/A'
_e = assert_raises(TypeError, read_key, d, 'debug', int)
assert_message_contains(_e, 'wrong type')
print('OK - 2')

OK - 2


## 3) Divide or Log (with `else`)
Implement `divide_or_log(a, b, logger)`:
- On success, append `'ok'` to `logger` in the `else:` block and return the result.
- On `ZeroDivisionError`, append a short message and return `float('inf')`.
- Do **not** catch other exceptions.

In [6]:
# TODO: implement divide_or_log
def divide_or_log(a, b, logger):
    try:
        res = a / b
    except ZeroDivisionError as e:
        logger.append(f'ZeroDivisionError: {e}')
        return float('inf')
    else:
        logger.append('ok')
        return res

divide_or_log(6, 3, [])

2.0

In [7]:
log = []
assert divide_or_log(8, 4, log) == 2.0
assert log[-1] == 'ok'
r = divide_or_log(1, 0, log)
assert r == float('inf') and any('ZeroDivisionError' in s for s in log)
print('OK - 3')

OK - 3


## 4) Sum Only Numeric Elements (skip others)
Implement `sum_numeric(iterable)` that adds numeric elements (int/float). If an element raises `TypeError` on `+`, **skip** it. If no numeric elements exist, return `0`.

> Use a `try/except` inside the loop to skip just the bad element.

In [8]:
# TODO: implement sum_numeric
def sum_numeric(iterable):
    total = 0
    count = 0
    for x in iterable:
        try:
            total += x
            count += 1
        except TypeError:
            pass
    return total if count > 0 else 0

sum_numeric([1, 'a', 2.5])

3.5

In [9]:
assert sum_numeric([1, 'a', 2.5]) == 3.5
assert sum_numeric([]) == 0
assert sum_numeric(['x']) == 0
print('OK - 4')

OK - 4


## 5) Ensure Cleanup with `finally`
Implement `use_resource(div_by)` that computes `10/div_by` but always marks the resource as closed using a `finally` block. Track the last-closed flag on the function itself as `use_resource.last_closed` for testing.

> Even if a `ZeroDivisionError` happens, `last_closed` must be `True` after the call exits.

In [10]:
# TODO: implement use_resource with finally
def use_resource(div_by):
    use_resource.last_closed = False
    class _Dummy:
        def close(self):
            use_resource.last_closed = True
    r = _Dummy()
    try:
        return 10 / div_by
    finally:
        r.close()

use_resource(2)

5.0

In [11]:
assert use_resource(2) == 5
assert use_resource.last_closed is True
try:
    use_resource(0)
except ZeroDivisionError:
    pass
assert use_resource.last_closed is True
print('OK - 5')

OK - 5


## 6) Parse JSON Field with Unified Error
Implement `parse_json_field(s, field)` that returns `json.loads(s)[field]`.
- On JSON errors, wrong type, or missing field, raise **one** `ValueError('invalid payload or missing field')` and chain the original (`from err`).

In [12]:
# TODO: implement parse_json_field with chaining
import json

def parse_json_field(s, field):
    try:
        obj = json.loads(s)
        return obj[field]
    except Exception as err:
        raise ValueError('invalid payload or missing field') from err

parse_json_field('{"x": 1}', 'x')

1

In [13]:
assert parse_json_field('{"n": 7}', 'n') == 7
_e = assert_raises(ValueError, parse_json_field, '{bad json', 'x')
assert _e.__cause__ is not None
_e = assert_raises(ValueError, parse_json_field, '{"x": 1}', 'y')
assert_message_contains(_e, 'invalid payload')
print('OK - 6')

OK - 6


## 7) Try Multiple Sources, Report Failures
Implement `fetch_first(urls, fetch)` that tries each `url` with `fetch(url)` until one succeeds, then returns its value.
- Collect messages of failures; if all fail, raise `RuntimeError('all fetches failed: ...')` summarizing the errors.
- Only catch broad exceptions if you **use them** (to record) and then **raise** your own summary error.

In [14]:
# TODO: implement fetch_first
def fetch_first(urls, fetch):
    failures = []
    for u in urls:
        try:
            return fetch(u)
        except Exception as e:
            failures.append(str(e))
    raise RuntimeError('all fetches failed: ' + '; '.join(failures))

def _fake_fetch(u):
    if 'ok' in u:
        return f'data:{u}'
    raise IOError(f'404 for {u}')

fetch_first(['a', 'b', 'ok'], _fake_fetch)

'data:ok'

In [15]:
assert fetch_first(['x', 'y', 'ok'], _fake_fetch) == 'data:ok'
_e = assert_raises(RuntimeError, fetch_first, ['x'], _fake_fetch)
assert_message_contains(_e, 'all fetches failed')
assert_message_contains(_e, '404 for x')
print('OK - 7')

OK - 7


## 8) Ignore Specific Errors with `contextlib.suppress`
Implement `count_valid(callables_list)` that calls each zero-arg function:
- Count how many calls succeed.
- Ignore (`suppress`) **only** `ValueError`.
- Let other exceptions propagate (e.g., `TypeError`).

In [16]:
# TODO: implement count_valid using suppress
from contextlib import suppress

def count_valid(callables_list):
    count = 0
    for fn in callables_list:
        with suppress(ValueError):
            fn()
            count += 1
    return count

def _ok():
    int('10')

def _bad_val():
    int('x')  # ValueError

def _bad_type():
    int(None)  # TypeError

count_valid([_ok, _bad_val, _ok])

2

In [17]:
assert count_valid([_ok, _bad_val, _ok]) == 2
_e = assert_raises(TypeError, count_valid, [_ok, _bad_type])
print('OK - 8')

OK - 8


## 9) Simple Retry Helper
Implement `try_n_times(n, func, *a, **kw)`:
- Call `func(*a, **kw)` up to `n` times until success.
- Catch any exception, keep the last, and after `n` failures re-raise the last one.
- Don’t swallow the error silently—only try again and surface it if all attempts fail.

In [18]:
# TODO: implement try_n_times
def try_n_times(n, func, *a, **kw):
    last_exc = None
    for _ in range(n):
        try:
            return func(*a, **kw)
        except Exception as e:
            last_exc = e
    raise last_exc

def make_flaky(k, result):
    calls = {'n': 0}
    def f():
        calls['n'] += 1
        if calls['n'] <= k:
            raise RuntimeError('flaky')
        return result
    return f

try_n_times(3, make_flaky(2, 'OK'))

'OK'

In [19]:
assert try_n_times(3, make_flaky(2, 99)) == 99
_e = assert_raises(RuntimeError, try_n_times, 2, make_flaky(3, 'never'))
assert_message_contains(_e, 'flaky')
print('OK - 9')

OK - 9


## 10) `try/except/else` for Validation
Implement `all_parsable_ints(seq)` that returns `True` iff **every** element in `seq` can be converted by `int()`.
- Use one `try` around the whole loop and put the loop in the `try` block.
- On any failure, return `False` from `except`.
- If no exception, return `True` from `else`.

In [20]:
# TODO: implement all_parsable_ints using try/except/else
def all_parsable_ints(seq):
    try:
        for x in seq:
            int(x)
    except Exception:
        return False
    else:
        return True

all_parsable_ints(['1', 2, 3.0])

True

In [21]:
assert all_parsable_ints(['1', 2, 3.0]) is True
assert all_parsable_ints(['1', 'x']) is False
assert all_parsable_ints([]) is True
print('OK - 10')

OK - 10
